In [ ]:
"""
Grid preparation for future aspen project:

Author: maxwell.cook@colorado.edu
"""

import os, sys, time, re
import pandas as pd
import xarray as xr
import geopandas as gpd
import numpy as np
import rasterio as rio
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

from matplotlib.ticker import FuncFormatter
from tqdm.notebook import tqdm
from matplotlib.colors import to_rgba
from matplotlib.patches import Patch
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import distance_transform_edt

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/')

# Custom functions
sys.path.append(os.path.join(maindir,'aspen-fire/Aim2/code/Python'))
from __functions import *

proj = 'EPSG:5070' # albers

print("Ready to go !")

## Future Fire

From Stephens et al., In review, annual probability of area burned and fire occurrence. Trend to 2060 was calculated using a median-based linear model (MBLM) thielsen estimate. 

In [ ]:
fp = os.path.join(projdir,'Aim3/data/spatial/mod/future_fire_grid_trend.gpkg')
future_fire = gpd.read_file(fp)
future_fire.head()

In [ ]:
# check for duplicates, remove them
n = future_fire.duplicated(subset=['grid_id']).sum()
if n > 0:
    print(f"\nThere are [{n}] duplicate rows.\n")
else:
    print("\nNo duplicates at this stage.\n")

## Aspen habitat suitability 

From Hart et al. In Review- Calculate the change between past and future (2041-2070) aspen suitability to get the difference (future change in habitat suitability).

In [ ]:
# load the future aspen suitability predictions
dir = os.path.join(projdir,'Aim3/data/spatial/raw/aspen_suitability/Predictions/')
# historical suitability
hist = os.path.join(dir,'predicted_historical_1981-2010_ensemble.tif')
# future suitability (SSP245 2041-2070)
ssp245 = os.path.join(dir,'predicted_SSP245_2041-2070_ensemble.tif')
# future suitability (SSP585 2041-2070)
ssp585 = os.path.join(dir,'predicted_SSP585_2041-2070_ensemble.tif')

# load each raster
hist_da = rxr.open_rasterio(hist, masked=True, cache=False).squeeze()
ssp245_da = rxr.open_rasterio(ssp245, masked=True, cache=False).squeeze()
ssp585_da = rxr.open_rasterio(ssp585, masked=True, cache=False).squeeze()

# calculate the change in suitability under both scenarios
change_SSP245 = ssp245_da - hist_da
change_SSP585 = ssp585_da - hist_da

# check the results
out_png = os.path.join(projdir, 'Aim3/figures/srm_aspen_suitability_change_ssp585.png')
plot_raster(change_SSP585, legend_lab = "Aspen Suitability Change (SSP585)", save_file=True, out_png=out_png)

In [ ]:
change_SSP245.rio.crs

In [ ]:
# calculate band statistics
# mean, standard deviation, percentiles

# create a list of rasters to calculate zonal stats
das = {
    'historic': hist_da, # historic (1981-2010) suitability
    'ssp245': ssp245_da, # raw suitability SSP245 2041-2070
    'ssp585': ssp585_da, # raw suitability SSP585 2041-2070
    'delta245': change_SSP245, # change SSP245 2041-2070
    'delta585': change_SSP585 # change SSP585 2041-2070
}

results = []
for key, da in das.items():
    print(f"Processing: {key}")
    # ensure the correct projection
    da_ = da.rio.reproject(proj) # matches grid
    # calculate zonal statistics
    zs = compute_band_stats(
        geoms=future_fire, 
        image_da=da_, 
        id_col='grid_id', 
        stats=['mean'], # 'median','std','percentile_90'
        attr=key,
        ztype='continuous'
    )
    results.append(zs)

# concatenate the results
future_aspen = pd.concat([df.set_index("grid_id") for df in results], axis=1).reset_index()
future_aspen.head()

In [ ]:
# rename the columns
future_aspen = future_aspen.rename(
    columns={
        'historic_mean': 'historic',
        'ssp245_mean': 'ssp245',
        'ssp585_mean': 'ssp585',
        'delta245_mean': 'delta245',
        'delta585_mean': 'delta585'
    }
)

# join to the fire dataframe
fire_fa = pd.merge(future_fire, future_aspen, on="grid_id", how="left")
fire_fa.head()

# save this file out
out_fp = os.path.join(projdir, 'Aim3/data/spatial/mod/future_fire_grid_trend_aspen.gpkg')
fire_fa.to_file(out_fp)
print(f"Saved to: \n{out_fp}")

In [ ]:
# tidy up
del das, results, zs, hist_da, ssp245_da, ssp585_da, change_SSP245, change_SSP585, future_aspen
gc.collect()

## Sentinel-based aspen cover (ca. 2019)

From Cook et al. (2024), calculate the percent aspen canopy cover within gridcells. This should ideally be updated to 2023 ...

In [ ]:
t0 = time.time()

# load the 10-m aspen map (classification)
aspen10_fp = os.path.join(projdir,'Aim1/data/spatial/mod/results/classification/s2aspen_distribution_10m_y2019_CookEtAl.tif')
aspen10 = rxr.open_rasterio(aspen10_fp, cache=False, chunks='auto', mask=True).squeeze()
print(f"\n{aspen10}\n")
print(aspen10.rio.crs)

# calculate zonal statistics
aspen10_grids = compute_band_stats(
    geoms=future_fire, 
    image_da=aspen10, 
    id_col='grid_id', 
    attr='aspen10',
    ztype='categorical'
)

# only keep the count of aspen pixels
aspen10_grids = aspen10_grids[aspen10_grids['aspen10'] == 1]
# check the results
print(aspen10_grids.head())

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.\n")
print("\n~~~~~~~~~~\n")

In [ ]:
aspen10_grids['pct_cover'].describe()

In [ ]:
# rename for clarity
aspen10_grids.rename(columns={
    'pct_cover': 'aspen10_pct',
    'count': 'aspen10_pixn'
}, inplace=True)

# merge back to the main grid data
fire_fa_ = pd.merge(fire_fa, aspen10_grids[['grid_id','aspen10_pct','aspen10_pixn']], on='grid_id')
fire_fa_.columns

In [ ]:
del aspen10, aspen10_grids, fire_fa
gc.collect() # free memory

## Built Environment Summaries

COMBUST, Microsoft Building Footprints, WorldPop population density/counts, and WUI classification (10-m)

In [ ]:
# Contemporary COMBUST grid (combustible mass of the built environment)
combust_fp = os.path.join(projdir,'Aim3/data/spatial/raw/COMBUST/combustible_building_mass_t_2020_mean.tif')
# Miscrosoft building footprint counts (same grid as COMBUST)
msbf_fp = os.path.join(maindir,'earth-lab/opp-urban-fuels/data/spatial/mod/Microsoft/msbf_counts.tif')
# WorldPop population density estimates
wp_density_fp = os.path.join(projdir,'Aim3/data/spatial/raw/WorldPop/usa_pd_2020_1km.tif')
wp_count_fp = os.path.join(projdir,'Aim3/data/spatial/raw/WorldPop/srm_ppp_2020_constrained.tif')

# put the continuous rasters in a dictionary to calculate zonal stats
das = {
    'combust': combust_fp,
    'msbf_count': msbf_fp,
    'pop_density': wp_density_fp,
    'pop_count': wp_count_fp,
}

results = []
# map the dictionary, calculate zonal stats
for key, fp in das.items():
    print(f"Processing: {key}")
    # process the raster grid
    da = rxr.open_rasterio(fp, masked=True, cache=False).squeeze()
    da = da.rio.reproject(proj) # ensure matching projection
    if key == 'pop_density':
        stat = ['mean']
    else:
        stat = ['sum']
    # calculate zonal statistics
    zs = compute_band_stats(
        geoms=future_fire, 
        image_da=da, 
        id_col='grid_id', 
        stats=stat, 
        attr=key,
        ztype='continuous'
    )
    results.append(zs)
    del da, zs
    
# concatenate the results
built_env_pop = pd.concat([df.set_index("grid_id") for df in results], axis=1).reset_index()
built_env_pop.head()

### Wildland Urban Interface/Intermix (SILVIS)

In [1]:
# load the results from "wui.ipynb"
fp = os.path.join(projdir, 'Aim3/data/tabular/srm_wui_grid_stats.csv')
wui = pd.read_csv(fp)
wui.head()

NameError: name 'os' is not defined

In [ ]:
# merge the built environment stats with the wui stats


## Contemporary Fire Activity and Fire Risk 

### Ignition Causes

In [ ]:
# load the fpa-fod ignitions data and get the total count
fpa_fp = os.path.join(projdir,'Aim3/data/spatial/mod/fire/srm_fpa_fod.gpkg')
fpa = gpd.read_file(fpa_fp)
fpa.columns

In [ ]:
print(fpa['FIRE_YEAR'].min())
print(fpa['FIRE_YEAR'].max())

In [ ]:
# load the ICS (for 2021-2023)

In [ ]:
fpa['NWCG_CAUSE_CLASSIFICATION'].unique()

In [ ]:
# calculate the area burned by ignition cause
cause_burned = (
    fpa.groupby('NWCG_CAUSE_CLASSIFICATION', as_index=False)
    .agg(
        area_burned=('FIRE_SIZE','sum'),
        fire_count=('OBJECTID', 'count')   
    )
)
# rename the cause column
cause_burned.rename(columns={'NWCG_CAUSE_CLASSIFICATION': 'cause'}, inplace=True)
# replace the undetermined code
cause_burned['cause'] = cause_burned['cause'].replace(
    "Missing data/not specified/undetermined", "Undetermined"
)
# check the results
cause_burned.head()

In [ ]:
# Define labels and values for both metrics
labels = cause_burned['cause']
sizes_area = cause_burned['area_burned']
sizes_count = cause_burned['fire_count']

# define color mapping
color_map = {
    "Human": "#B22222",        
    "Natural": "#377EB8",      
    "Undetermined": "#B3B3B3"  
}
# map the colors
colors = [color_map.get(cause, "#CCCCCC") for cause in cause_burned["cause"]]

# Create subplots for two pie charts
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

axes[0].pie(sizes_area, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
axes[0].text(-1.3, 1.1, "A", fontsize=11, fontweight="bold")  # Add subplot label "A"
axes[1].pie(sizes_count, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
axes[1].text(-1.3, 1.1, "B", fontsize=11, fontweight="bold")  # Add subplot label "B"

# Show plot
plt.tight_layout()

plt.savefig(os.path.join(projdir, 'Aim3/figures/FPA-FOD_cause_pie.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
del fpa

### Burned Area

In [ ]:
# load burned area and calculate the cumulative area burned
mtbs_fp = os.path.join(projdir,'Aim3/data/spatial/mod/fire/srm_mtbs_perims_dd.gpkg')
mtbs = gpd.read_file(mtbs_fp)
mtbs.columns

In [ ]:
mtbs['Ig_Date'] = pd.to_datetime(mtbs['Ig_Date'])
mtbs['Fire_Year'] = mtbs['Ig_Date'].dt.year
mtbs[['Ig_Date','Fire_Year']].head()

In [ ]:
print(mtbs['Fire_Year'].min())
print(mtbs['Fire_Year'].max())

In [ ]:
# plot the annual area burned
burned_area = mtbs.groupby('Fire_Year')['BurnBndAc'].sum().reset_index()

# Function to format colorbar labels with 'k' notation
def format_ticks(value, _):
    return f'{int(value / 1000)}k'

formatter = FuncFormatter(format_ticks)

# Plot the results
plt.figure(figsize=(7, 3))
plt.bar(burned_area['Fire_Year'], burned_area['BurnBndAc'], color='grey', edgecolor='black')
plt.xlabel('Year', fontsize=10)
plt.ylabel('Area Burned (ha)', fontsize=9)
plt.xticks(burned_area['Fire_Year'], rotation=45, size=8)

# Apply custom formatting to y-axis
plt.gca().yaxis.set_major_formatter(formatter)
# plt.ylim(10000, None)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig(os.path.join(projdir, 'Aim3/figures/MTBS_annualBurnedArea.png'), dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# calculate the cumulative previous area burned
mtbs = mtbs.to_crs(proj) # ensure the correct CRS
mtbs_in_grid = gpd.overlay(mtbs, future_fire, how='intersection') # perform the overlay
# get the cumulative burned area in each grid cell
mtbs_in_grid['burned_area'] = mtbs_in_grid.geometry.area
burned_area_c = mtbs_in_grid.groupby('grid_id', as_index=False)['burned_area'].sum()
# calculate the cumulative percent burned
# get the gridcell area
future_fire['grid_area'] = future_fire.geometry.area

# merge burned area summary back to grid cells
future_fire_ = future_fire.merge(burned_area_c, on='grid_id', how='left')
# fill NaN values (for grid cells with no fire) with 0
future_fire_['burned_area'] = future_fire_['burned_area'].fillna(0)
# Compute percentage of each grid cell burned
future_fire_['burned_pct'] = (future_fire_['burned_area'] / future_fire_['grid_area']) * 100
future_fire_ = future_fire_[['grid_id','burned_area','burned_pct']]
print(future_fire_.head())

# merge back to the main grid data
fire_fa_blt_ba = pd.merge(fire_fa_blt, future_fire_, on='grid_id', how='left')
fire_fa_blt_ba.columns

In [ ]:
del mtbs

### Wildfire Hazard Potential (version 2023)

In [ ]:
# load the wildfire hazard potential
fp = os.path.join(projdir,'Aim3/data/spatial/raw/fsim/Data/whp2023_GeoTIF/whp2023_cnt_conus.tif')
whp = rxr.open_rasterio(fp, masked=True, chunks='auto').squeeze()
whp = whp.rio.reproject(proj) # matches grid
# calculate the zonal stats
whp_zs = compute_band_stats(
    geoms=future_fire, 
    image_da=whp, 
    id_col='grid_id', 
    stats=['percentile_90'], # 'median','std','percentile_90'
    attr='whp',
    ztype='continuous'
)
whp_zs.head()

In [ ]:
# merge back to the main grid data
whp_zs.rename(columns={'whp_percentile_90': 'whp_p90'}, inplace = True) # tidy columns name first
fire_fa_blt_ba_ = pd.merge(fire_fa_blt_ba, whp_zs, on='grid_id', how='left')
fire_fa_blt_ba_.columns

## LANDFIRE Existing Vegetation (ca. 2023)

In [ ]:
# list out the landfire GeoTIFFs
# veg Type, Height, and Cover
lf_dir = os.path.join(maindir,'Aim3/data/spatial/mod/landfire/')
lf_tiffs = list_files(lf_dir, '*.tif', recursive=True)
print([os.path.basename(f) for f in lf_tiffs])

In [ ]:
# calculate the percent cover from EVT classes
evt = rxr.open_rasterio(lf_tiffs[2], masked=True).squeeze()
print(evt.rio.crs)

# calculate the percent cover of WUI classes for each grid
t0 = time.time()

# see __functions.py
evt_grid = compute_band_stats(future_fire, evt, 'grid_id', attr='evt')
# tidy columns in the summary table
evt_grid['count'] = evt_grid['count'].astype(int)
evt_grid['total_pixels'] = evt_grid['total_pixels'].astype(int)
evt_grid.rename(columns = {'count': 'evt_pixels'}, inplace=True)

print(evt_grid.head())

t1 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t1:.2f} minutes.")
print("\n~~~~~~~~~~\n")

del evt # clean up 
gc.collect()

In [ ]:
# calculate the continuous summaries (EVC and EVH)